In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/roberta-base-save/rob_tok.zip
/kaggle/input/roberta-base-save/__results__.html
/kaggle/input/roberta-base-save/rob.zip
/kaggle/input/roberta-base-save/__notebook__.ipynb
/kaggle/input/roberta-base-save/__output__.json
/kaggle/input/roberta-base-save/custom.css
/kaggle/input/commonlitreadabilityprize/sample_submission.csv
/kaggle/input/commonlitreadabilityprize/train.csv
/kaggle/input/commonlitreadabilityprize/test.csv


In [2]:
import numpy as np
import pandas as pd
from transformers import RobertaTokenizer, RobertaModel, AdamW, get_linear_schedule_with_warmup
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import StratifiedKFold
import xgboost as xgb
import tqdm
from matplotlib import pyplot as plt
import copy
import gc
import pickle
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope as ho_scope

In [3]:
%%bash
cp ../input/roberta-base-save/rob.zip .
cp ../input/roberta-base-save/rob_tok.zip .
unzip rob.zip
unzip rob_tok.zip 
rm -r rob.zip rob_tok.zip

Archive:  rob.zip
   creating: rob/
  inflating: rob/pytorch_model.bin   
  inflating: rob/config.json         
Archive:  rob_tok.zip
   creating: rob_tok/
  inflating: rob_tok/vocab.json      
  inflating: rob_tok/special_tokens_map.json  
  inflating: rob_tok/tokenizer_config.json  
  inflating: rob_tok/merges.txt      


In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [5]:
data = pd.read_csv('/kaggle/input/commonlitreadabilityprize/train.csv')
data.head()

,id,url_legal,license,excerpt,target,standard_error
0,c12129c31,NaN,NaN,When the young people returned to the ballroom...,-0.340259,0.464009
1,85aa80a4c,NaN,NaN,"All through dinner time, Mrs. Fayre was somewh...",-0.315372,0.480805
2,b69ac6792,NaN,NaN,"As Roger had predicted, the snow departed as q...",-0.580118,0.476676
3,dd1000b26,NaN,NaN,And outside before the palace a great garden w...,-1.054013,0.450007
4,37c1b32fb,NaN,NaN,Once upon a time there were Three Bears who li...,0.247197,0.510845


In [6]:
class ContinuousStratifiedKFold(StratifiedKFold):
    def split(self, X, y, groups=None):
        num_bins = int(np.floor(1 + np.log2(len(y))))
        bins = pd.cut(y, bins=num_bins, labels=False)
        return super().split(X, bins, groups)

class BERTRegressorMP(torch.nn.Module): 
    def __init__(self, pretrained_src = 'rob'): 
        super().__init__()
        self.bert = RobertaModel.from_pretrained(pretrained_src)
        self.linear = torch.nn.Linear(768, 1)
        self.dropout = torch.nn.Dropout(0.1)        
        
    def forward(self, input_ids, attention_mask):           
        last_hidden_state = self.bert(input_ids, 
                           attention_mask=attention_mask)[0]   
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
        sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, 1)
        sum_mask = input_mask_expanded.sum(1)
        sum_mask = torch.clamp(sum_mask, min=1e-9)
        mean_embeddings = sum_embeddings / sum_mask
        output = self.linear(self.dropout(mean_embeddings))
        return output
    
class EmbedderMP(torch.nn.Module):
    def __init__(self, model): 
        super().__init__()
        self.bert = model.bert
    
    def forward(self, input_ids, attention_mask): 
        last_hidden_state = self.bert(input_ids, 
                           attention_mask=attention_mask)[0]   
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
        sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, 1)
        sum_mask = input_mask_expanded.sum(1)
        sum_mask = torch.clamp(sum_mask, min=1e-9)
        mean_embeddings = sum_embeddings / sum_mask
        return mean_embeddings

class RMSELoss(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.mse = torch.nn.MSELoss()
        
    def forward(self,yhat,y):
        loss = torch.sqrt(self.mse(yhat,y))
        return loss

def rmse_metric(y_true, y_pred):
    return np.sqrt(mse(y_true, y_pred))


def evaluate(dataloader_val, model):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in tqdm.notebook.tqdm(dataloader_val):
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1]
        }
        target = batch[2]

        with torch.no_grad():        
            output = model(**inputs)
            
        loss = criterion(output, target.view(-1,1))
        loss_val_total += loss

        output = output.detach().cpu().numpy()
        target = target.cpu().numpy()
        predictions.append(output)
        true_vals.append(target)
    
    loss_val_avg = loss_val_total / len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals


def get_bert_embeddings(embedder, dataloader, device = device):
    embedder.eval()
    embeddings_all = []
    for batch in tqdm.notebook.tqdm(dataloader):        
        batch = tuple(b.to(device) for b in batch)        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1]
                 }
        with torch.no_grad():        
            output = embedder(**inputs)

        embeddings_batch = output.detach().cpu().numpy()
        embeddings_all.append(embeddings_batch)

    return np.vstack(embeddings_all)

In [7]:
tokenizer = RobertaTokenizer.from_pretrained(
    'rob_tok'
)

In [8]:
BATCH_SIZE = 16

warm_prop = 0.1
epochs = 8
clip = 1

In [9]:
kf = ContinuousStratifiedKFold(n_splits=5, shuffle=True, random_state=42)

print('START')
for k, (train_ids, val_ids) in enumerate(tqdm.notebook.tqdm(kf.split(X=data, y=data['target'].values))):
    print('**************')
    print('------------')
    print('**************')
    print(f'ITERATION {k} starts')

    gc.collect()

    print('------------')
    print('data slicing ...')

    data_train = data.iloc[train_ids]['excerpt'].values
    data_val = data.iloc[val_ids]['excerpt'].values
    print(f'train/val data shapes: {data_train.shape}, {data_val.shape}')
    target_train = data.iloc[train_ids]['target'].values
    target_val = data.iloc[val_ids]['target'].values


    print('------------')
    print('data preparation ...')
    encoded_data_train = tokenizer.batch_encode_plus(
        data_train,
        add_special_tokens=True,
        return_attention_mask=True,
        pad_to_max_length=True,
        max_length=512,
        return_tensors='pt',
    )

    encoded_data_val = tokenizer.batch_encode_plus(
        data_val,
        add_special_tokens=True,
        return_attention_mask=True,
        pad_to_max_length=True,
        max_length=512,
        return_tensors='pt'
    )


    input_ids_train = encoded_data_train['input_ids']
    attention_masks_train = encoded_data_train['attention_mask']
    values_train = torch.tensor(target_train, dtype=torch.float)

    input_ids_val = encoded_data_val['input_ids']
    attention_masks_val = encoded_data_val['attention_mask']
    values_val = torch.tensor(target_val, dtype=torch.float)

    dataset_train = TensorDataset(input_ids_train,
                                 attention_masks_train,
                                 values_train)
    dataset_val = TensorDataset(input_ids_val,
                                attention_masks_val,
                                values_val)

    dataloader_train = DataLoader(
        dataset_train,
        sampler=RandomSampler(dataset_train),
        batch_size=BATCH_SIZE
    )

    dataloader_val = DataLoader(
        dataset_val,
        sampler=RandomSampler(dataset_val),
        batch_size=2*BATCH_SIZE
    )

    print('------------')
    print('Roberta finetuning ...')

    model = BERTRegressorMP().to(device)
    criterion = RMSELoss()
    optimizer = AdamW(
        model.parameters(),
        lr= 3e-5,#the original paper:2e-5 -> 5e-5
        eps=1e-8
    )
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=int(len(dataloader_train)*epochs * warm_prop),
        num_training_steps=len(dataloader_train)*epochs
    )


    best_val_loss = float('inf')
    for epoch in tqdm.notebook.tqdm(range(epochs)):
        print(f'Epoch {epoch}')
        model.train()

        epoch_loss = 0
        for batch in tqdm.notebook.tqdm(dataloader_train):

            batch = tuple(b.to(device) for b in batch)
            inputs = {'input_ids':      batch[0],
                    'attention_mask': batch[1]
              }
            target = batch[2]

            optimizer.zero_grad()        

            output = model(**inputs)     
            loss = criterion(output, target.view(-1,1))      
            loss.backward()
            epoch_loss += loss.item()

            torch.nn.utils.clip_grad_norm_(model.parameters(), clip)      
            optimizer.step()
            scheduler.step()     

        val_loss, predictions, true_vals = evaluate(dataloader_val, model)
        if val_loss < best_val_loss:
            best_val_loss = val_loss        
            torch.save(model.state_dict(), f'roberta_base_{k}.pt')
            best_model = copy.deepcopy(model).cpu()
        train_loss = epoch_loss / len(dataloader_train)
        rmse_val = rmse_metric(true_vals, predictions)
        print('-------')
        print(f'Training loss: {train_loss}')
        print(f'Validation loss: {val_loss}')
        print(f"RMSE on validation: {rmse_val}")
    
    print('loaded best model with lm head performance...')
    dataloader_val_frozen = DataLoader(
        dataset_val,
        batch_size=2*BATCH_SIZE
    )  
    
    model = BERTRegressorMP()
    PATH = f'roberta_base_{k}.pt'
    model.load_state_dict(torch.load(PATH), strict=False)
    model.to(device)
    val_loss, predictions, true_vals = evaluate(dataloader_val_frozen, model)
    rmse_val = rmse_metric(true_vals, predictions)
    print(f"RMSE on validation: {rmse_val}")
    
    del model
    gc.collect()

    print('------------')
    print('Embeddings extraction ...')

    embedder = EmbedderMP(best_model).to(device)    
    
    dataloader_train_frozen = DataLoader(
        dataset_train,
        batch_size=2*BATCH_SIZE,
    )

    embeddings_train = get_bert_embeddings(embedder, dataloader_train_frozen)
    embeddings_val = get_bert_embeddings(embedder, dataloader_val_frozen)
    
    del embedder
    gc.collect()
    
    print('------------')
    print('XGB head HP tuning ...')
    def hyperopt_train_test(params):
        estimator = xgb.XGBRegressor(**params)  
        estimator.fit(embeddings_train, target_train)
        preds = estimator.predict(embeddings_val)
        metric = rmse_metric(target_val, preds)
        return metric

    space_xgb = {'n_estimators': ho_scope.int(hp.quniform('n_estimators', 50, 400, q=25)),
             'learning_rate': hp.loguniform('learning_rate', np.log(0.0001), np.log(0.5)) - 0.0001,
             'max_depth': ho_scope.int(hp.quniform('max_depth', 1, 11, 1)),
             'min_child_weight': ho_scope.int(hp.quniform('min_child_weight', 0, 300, 25)),
             'subsample': hp.quniform('subsample', 0.5, 1, 0.05),
             'gamma': hp.loguniform('gamma', np.log(0.0001), np.log(5)) - 0.0001,
             'colsample_bytree': hp.quniform('colsample_bytree', 0.5, 1, 0.05), 
             'colsample_bylevel': hp.quniform('colsample_bylevel', 0.5, 1, 0.05),
             'reg_alpha': hp.loguniform('reg_alpha', np.log(0.0001), np.log(1)) - 0.0001,
             'reg_lambda': hp.loguniform('reg_lambda', np.log(1), np.log(4))
             }
    def f(params):
        rmse_metric_val = hyperopt_train_test(params)
        return {'loss': rmse_metric_val, 'status': STATUS_OK}

    trials = Trials()
    best = fmin(f, space_xgb, algo = tpe.suggest, max_evals = 50, trials=trials)
    print('best', best)

    best['max_depth'] = int(best['max_depth'])
    best['min_child_weight'] = int(best['min_child_weight'])
    best['n_estimators'] = int(best['n_estimators'])
    best['random_state'] = 42
    best['n_jobs'] = -1
    best['objective'] = 'reg:squarederror'
    xgb_head = xgb.XGBRegressor(**best)    
    xgb_head.fit(embeddings_train, target_train)
    filename = f'xgb_head_{k}.pkl'
    pickle.dump(xgb_head, open(filename, 'wb'))
    xgb_head = pickle.load(open(filename, 'rb'))
    preds_xgb = xgb_head.predict(embeddings_val)
    rmse_val = rmse_metric(target_val, preds_xgb)
    print(f"RMSE on validation: {rmse_val}")    

    print('------------')
    print(f'Iteration {k} completed.')
    
print('**************')
print('------------')
print('**************')    
print('FINISH')

START


0it [00:00, ?it/s]

**************
------------
**************
ITERATION 0 starts
------------
data slicing ...


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


train/val data shapes: (2267,), (567,)
------------
data preparation ...


...

  0%|          | 0/18 [00:00<?, ?it/s]

RMSE on validation: 0.5280457735061646
------------
Embeddings extraction ...


  0%|          | 0/71 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

------------
XGB head HP tuning ...

100%|██████████| 50/50 [12:55<00:00, 15.52s/trial, best loss: 0.5024177888952598]
best {'colsample_bylevel': 0.75, 'colsample_bytree': 0.5, 'gamma': 0.0070726847630180785, 'learning_rate': 0.03353042746762655, 'max_depth': 5.0, 'min_child_weight': 275.0, 'n_estimators': 400.0, 'reg_alpha': 0.8752739661656103, 'reg_lambda': 1.3743327240074756, 'subsample': 0.7000000000000001}
RMSE on validation: 0.49883397785339345
------------
Iteration 4 completed.
**************
------------
**************
FINISH


In [10]:
!rm -r rob rob_tok